In [1]:
from algo import *

In [28]:
import numpy as np
from sklearn.decomposition import NMF

def randomized_nmf(A, r, r_ov=5, w=1, compression='standard'):
    """
    NMF with compression matrices L and R, then using scipy's NMF to factorize 
    the compressed matrices A_hat = L^T A and A_check = A R^T.
    """
    m, n = A.shape
    d = r + r_ov  # Effective reduced dimension

    # --- Step 1: Compute compression matrices L and R ---
    if compression == 'standard':
        L = standard_compression(A, r, r_ov, w)      # m × d
        R = standard_compression(A.T, r, r_ov, w).T  # d × n
    elif compression == 'srht':
        L = srht_compression(A, r, r_ov, w)      # m × d
        R = srht_compression(A.T, r, r_ov, w).T  # d × n
    elif compression == 'fjlt':
        L = fjlt_compression(A, r, r_ov, w)      # m × d
        R = fjlt_compression(A.T, r, r_ov, w).T  # d × n
    elif compression == 'cwt':
        L = cwt_compression(A, r, r_ov, w)      # m × d
        R = cwt_compression(A.T, r, r_ov, w).T  # d × n
    else:
        raise ValueError('Please select a valid compression algorithm')

    # --- Step 2: Compute compressed matrices ---
    A_hat = L.T @ A  # d × n
    A_check = A @ R.T  # m × d

    # --- Step 3: Apply NMF on A_hat and A_check separately ---
    nmf1 = NMF(n_components=r, init='nndsvd', max_iter=500, tol=1e-6)
    W_hat = nmf1.fit_transform(A_hat)  # d × r
    H_hat = nmf1.components_  # r × n

    nmf2 = NMF(n_components=r, init='nndsvd', max_iter=500, tol=1e-6)
    W_check = nmf2.fit_transform(A_check)  # m × r
    H_check = nmf2.components_  # r × d

    # --- Step 4: Expand back to original space ---
    X_k = L @ W_hat  # m × r
    Y_k = H_check @ R  # r × n

    return X_k, Y_k, (nmf1.reconstruction_err_, nmf2.reconstruction_err_)

In [29]:
A,_,_ = generate_dense_matrix(1000,5,1)
A = np.abs(A)

In [30]:
randomized_nmf(A,5)

ValueError: Negative values in data passed to NMF initialization.